# QA Training Points

# Create LULC training data from existing LULC products.

Key considerations:
* Spatial distribution, especially in SIDS
* Class distribution
* Temporal coverage
* Quality and confidence
* Trends in agreement with LULC datasets

References
- https://github.com/frontiersi/FAO_LC_workshop_Rwanda/blob/main/0_Generate_Training_Points.ipynb
- https://github.com/frontiersi/land_cover_mapping_DEAfrica/blob/main/notebooks/Lesotho/Filter_training_data_GEE_replicate.ipynb
- https://github.com/auspatious/ldn-lulc/blob/main/notebooks/Compare_LULC.ipynb
- https://github.com/auspatious/ldn-lulc/blob/main/notebooks/Compare_LULC_per_class.ipynb

Steps
1. Extract points with reliable LU/LC classes from existing products. Use these 3 https://github.com/auspatious/ldn-lulc/blob/main/notebooks/Compare_LULC_per_class.ipynb. ESA Worldcover is the best one. Do this for Singaport or Suva, Fiji bounding box (due to presence of all classes hopefully). Classes should have roughly equal proportions in the sample points. Random sample all classes, use cluster method to reject outliers. This tells you which training points are more reliable.
2. Align typologies/classes from those products to our classes defined here (level 1 only) https://github.com/auspatious/ldn-lulc/blob/8dd22d4190207c977c55e4cde9f236df6daf8722/typology/typology.md
3. Extract the observational data (RGB and all other bands incl. geomad attributes), indices (NDVI, etc. TBC), DEM, etc.) This partially relies on geomad.
4. Then iterate and tune the method to extract reliable training data.

Output schema of training data: coordinates, crs, year, class, source of class e.g. "ESA Worldcover", RGB, all other bands, geomad properties, indices (e.g. NDVI), elevation.

Forked from https://github.com/frontiersi/FAO_LC_workshop_Rwanda/blob/main/0_Generate_Training_Points.ipynb and https://github.com/auspatious/ldn-lulc/blob/main/notebooks/Compare_LULC_per_class.ipynb

## Background

**Training data** is the most important part of any supervised machine learning workflow. The quality of the training data has a greater impact on the classification than the algorithm used. Large and accurate training data sets are preferable: increasing the training sample size results in increased classification accuracy ([Maxell et al 2018](https://www.tandfonline.com/doi/full/10.1080/01431161.2018.1433343)).  A review of training data methods in the context of Earth Observation is available [here](https://www.mdpi.com/2072-4292/12/6/1034).

There are many platforms to use for gathering land cover training labels, the best one to use depends on your application. GIS platforms are great for collecting training data as they are highly flexible and mature platforms; [Geo-Wiki](https://www.geo-wiki.org/) and [Collect Earth Online](https://collect.earth/home) are two open-source websites that may also be useful depending on the reference data strategy employed. Alternatively, there are many pre-existing training datasets on the web that may be useful, e.g. [Radiant Earth](https://www.radiant.earth/) manages a growing number of reference datasets for use by anyone. With locations of land cover labels available, we can extract features at these locations from satellite imagery as input for machine learning.  

## Description

As timely training data is not always available, in this notebook we demonstrate how to generate a set of randomly distributed training points for a selected AOI from an existing classification raster.

The workflow includes the following steps:

1. Preview the AOI (Singapore) on a basemap. Define datetime/year of interest.
2. Use 3 existing LU/LC products. Only use data where they agree. 30m resolution.
4. Remap/merge classes on the classification raster to our typologies.
5. Generate randomly distributed training points and export for future use.
6. Search and load GeoMAD, elevation.
7. Scale GeoMedian values and calculate indices.
8. Add GeoMAD, elevation and indices data to training data points. Export.



### Load packages


In [ ]:
# Reload functions during development
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import json

import numpy as np
import pandas as pd
import rasterio
import rioxarray  # noqa: F401
import xarray as xr
from matplotlib import pyplot as plt
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib.patches import Patch
from odc.geo.geom import BoundingBox, Geometry
from odc.stac import load
from planetary_computer import sign_url
from pystac import Item
from pystac.client import Client
from rustac import search_sync
from scipy.ndimage import uniform_filter

from ldn.random_sampling import random_sampling  # adapted from function by Chad Burton: https://gist.github.com/cbur24/04760d645aa123a3b1817b07786e7d9f
from ldn.typology import classes, colors, world_cover_map, cci_lc_map, io_map
from notebooks.src.Compare_LULC_func import standardise_class

In [ ]:
datetime = '2020-06'
datetime_year = datetime.split("-")[0]
lulc_resolution = 30 # meters. CCI: 300m, WC: 10m, IO: 10m. Meet in the middle with 30m.
lulc_class_raster = f'singapore_agreeing_lulc_{datetime}_{lulc_resolution}m.tif'
analysis_crs = 'EPSG:6933' # Equal Earth for global analysis
output_crs = 'EPSG:4326' # WGS84
# output_crs = 'EPSG:32648' # UTM Zone 48N for Singapore
class_attr='lulc'

bbox = BoundingBox(
    # TODO: Viti Levu, Fiji
    
    # Singapore
    left=103.6,
    bottom=1.2,
    right=104.1,
    top=1.48,
    crs=output_crs,
)
bbox.explore()

In [ ]:
# # Clip AOI bounding box to land. Too many ocean water points in the training data that are not relevant for LU/LC (and confuse the terrestrial water).
# # TODO: Find a more general way to get land for clipping. This only works for Singapore.
# import requests
# import geopandas as gpd
# from shapely.geometry import box
# from io import BytesIO

# # 1. Download GeoJSON
# url = "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_SGP_0.json"
# response = requests.get(url)
# response.raise_for_status()

# # 2. Read into GeoDataFrame
# gdf = gpd.read_file(BytesIO(response.content))

# # 3. Buffer 200m — project to a metric CRS first, then back
# gdf_projected = gdf.to_crs(analysis_crs)
# gdf_buffered = gdf_projected.copy()
# gdf_buffered["geometry"] = gdf_projected.buffer(200)
# gdf_buffered = gdf_buffered.to_crs(output_crs)

# # 4. Create bbox as MultiPolygon and clip to buffered land
# bbox_shape = box(bbox.left, bbox.bottom, bbox.right, bbox.top)
# bbox_gdf = gpd.GeoDataFrame(geometry=[bbox_shape], crs=output_crs)

# aoi = gpd.clip(bbox_gdf, gdf_buffered)

# aoi.explore()

In [ ]:
# This method of clipping the bbox to land works globally.

import geopandas as gpd
import duckdb
from shapely import wkt

con = duckdb.connect()
con.execute("INSTALL spatial; LOAD spatial;")

land_sgp = con.execute(f"""
    SELECT
        id,
        names['primary'] AS primary_name,
        ST_AsText(geometry) AS geometry
    FROM
        read_parquet('s3://overturemaps-us-west-2/release/2026-02-18.0/theme=base/type=land/*', filename=true, hive_partitioning=1)
    WHERE
        bbox.xmin >= {bbox.left}
        AND bbox.xmax <= {bbox.right}
        AND bbox.ymin >= {bbox.bottom}
        AND bbox.ymax <= {bbox.top}
""").df()

land_gdf = (
    gpd.GeoDataFrame(
        land_sgp,
        geometry=land_sgp['geometry'].apply(wkt.loads),
        crs="EPSG:4326"
    )
    .to_crs(analysis_crs)
)

# Buffer 200m and dissolve to clean up messy/overlapping polygons
land_buffered = (
    land_gdf
    .buffer(200)
    .union_all()
)

aoi = gpd.GeoDataFrame(geometry=[land_buffered], crs=analysis_crs)

In [ ]:
aoi.explore()
# print(aoi.to_json())


## Use 3 LU/LC products. Find agreement between them.

TODO: Experiment with data quality bands - some LULC datasets have quality information which could be used as additional filters.

In [ ]:
# Use Planetary Computer STAC API
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1/")
aoi_geojson = json.loads(aoi.to_json(to_wgs84=True))["features"][0]["geometry"]

# A generic function to load a product given the boundary
def load_lulc_data(product: str, resolution = 100):
    items = catalog.search(
        collections=[product],
        intersects=aoi_geojson,
        datetime=datetime,
    ).item_collection()

    print(f"Found {len(items)} items for product {product}")

    ds = load(
        items,
        intersects=aoi_geojson,
        crs=analysis_crs,
        groupby="solar_day",
        resolution=resolution,
        chunks={"x": 2048, "y": 2048},
        patch_url=sign_url,
        resample_alg="nearest",
    )

    print(f"Product {product} has variables: {ds.data_vars}")
    return ds

In [ ]:
cci_30m = load_lulc_data('esa-cci-lc', resolution=lulc_resolution) # 300m is native.
vars = list(cci_30m.data_vars)
cci_30m["esa_cci_lc"] = cci_30m["lccs_class"]
ds_tmp = cci_30m.drop_vars(vars)
cci_30m = ds_tmp.squeeze().load() # removes any singleton dimension and then load the full data

# Clip to AOI. ODC STAC Load intersects param doesn't clip.
cci_30m = cci_30m.rio.clip(aoi.to_crs(analysis_crs).geometry, crs=analysis_crs)

In [ ]:
cci_30m["esa_cci_lc"].odc.explore()
# cci_30m["esa_cci_lc"]

In [ ]:
wc_30m = load_lulc_data('esa-worldcover', resolution=lulc_resolution) # 10m is native.
vars = list(wc_30m.data_vars)
wc_30m["esa_worldcover"] = wc_30m["map"]
ds_tmp = wc_30m.drop_vars(vars)
wc_30m = ds_tmp.squeeze().load() #removes any singleton dimension and then load the full data

# Clip to AOI. ODC STAC Load intersects param doesn't clip.
wc_30m = wc_30m.rio.clip(aoi.to_crs(analysis_crs).geometry, crs=analysis_crs)

# Ensure alignment.
# TODO: Experiment with nearest resampling instead of majority vote
wc_30m["esa_worldcover"] = wc_30m["esa_worldcover"].rio.reproject_match(
    cci_30m["esa_cci_lc"], resampling=rasterio.enums.Resampling.mode  # majority vote (the value that appears most often)
)

In [ ]:
# wc_30m["esa_worldcover"].odc.explore()
wc_30m["esa_worldcover"]

In [ ]:
io_30m = load_lulc_data('io-lulc-annual-v02', resolution=lulc_resolution) # 10m is native.
vars = list(io_30m.data_vars)
io_30m["io_lulc"] = io_30m["data"]
ds_tmp = io_30m.drop_vars(vars)
io_30m = ds_tmp.squeeze().load() # removes any singleton dimension and then load the full data

# Clip to AOI. ODC STAC Load intersects param doesn't clip.
io_30m = io_30m.rio.clip(aoi.to_crs(analysis_crs).geometry, crs=analysis_crs)

# Ensure alignment.
# TODO: Experiment with nearest resampling instead of majority vote
io_30m["io_lulc"] = io_30m["io_lulc"].rio.reproject_match(
    cci_30m["esa_cci_lc"], resampling=rasterio.enums.Resampling.mode  # majority vote (the value that appears most often)
)

In [ ]:
io_30m["io_lulc"].odc.explore()

In [ ]:
# Standardise the classes to match the typology
cci_30m['esa_cci_lc'] = standardise_class(cci_30m['esa_cci_lc'], cci_lc_map)
wc_30m['esa_worldcover'] = standardise_class(wc_30m['esa_worldcover'], world_cover_map)
io_30m['io_lulc'] = standardise_class(io_30m['io_lulc'], io_map)
wc_30m

# Create layer where all 3 land cover products agree.

In [ ]:
agreement_threshold = 40

# TODO: Sample pixels with full agreement (and also their neighbors fully agree).
# Make a mask of pixels where all three products agree, then erode it by one pixel to ensure neighbors also agree, then sample from that.
# Current agreement selection is done on 5x5 grid.

# Valid pixels — all three products have data
valid = (wc_30m['esa_worldcover'] > 0) & (cci_30m['esa_cci_lc'] > 0) & (io_30m['io_lulc'] > 0)

# All-three agreement at each pixel (single boolean, no pairwise steps)
all_agree = (
    (wc_30m['esa_worldcover'] == cci_30m['esa_cci_lc']) &
    (cci_30m['esa_cci_lc'] == io_30m['io_lulc'])
).astype("float32").where(valid)

# 5x5 local agreement score
valid_f = valid.astype("float32").values
local_agree = xr.DataArray(
    np.where(
        (w := uniform_filter(valid_f, size=5, mode="nearest")) > 0,
        (uniform_filter(all_agree.fillna(0).values, size=5, mode="nearest") / w) * 100,
        np.nan,
    ),
    coords=all_agree.coords,
    dims=all_agree.dims,
    name="local_agreement_all_5x5",
)

# Output: agreed class value where threshold is met, else 0
agreed_class = wc_30m['esa_worldcover'].where(
    (local_agree >= agreement_threshold) & valid, other=0
).rename(class_attr)

_classes, counts = np.unique(agreed_class.values, return_counts=True)
counts_df = pd.DataFrame({"class": _classes, "pixel_count": counts})
print(counts_df)
print(agreed_class)

In [ ]:
# Write agreeing pixels as tiff.
agreed_class.rio.to_raster(lulc_class_raster, dtype='uint8', compress='deflate')

# # Read the tiff
# classification_map=xr.open_dataset(lulc_class_raster,engine="rasterio").astype(np.uint8)
# classification_map=classification_map.to_array().squeeze()
classification_map = agreed_class
classification_map

# Visualise the agreeing classes from 3 products

In [ ]:
fig, axes = plt.subplots(1,1)

# Plot classification map
unique_values=np.unique(classification_map)
cmap=ListedColormap([colors[k] for k in unique_values])
norm = BoundaryNorm(list(unique_values)+[np.max(unique_values)+1], cmap.N)
classification_map.plot.imshow(ax=axes, 
                   cmap=cmap,
                   norm=norm,
                   add_labels=True, 
                   add_colorbar=False,
                   interpolation='none')
# add colour legend
patches_list=[Patch(facecolor=colour) for colour in colors.values()]
axes.legend(patches_list, list(classes.keys()),loc='upper center', ncol =4, bbox_to_anchor=(0.5, -0.1))

## Generate random training samples
We generate some randomly distributed samples for each class from the clipped classification map using the `random_sampling` function. This function takes in a few parameters:  
* `da`: a classified map in the format of 2-dimensional xarray.DataArray
* `n`: total number of points to sample.
* `min_sample_n`: Minimum number of samples to generate per class if proportional number is smaller than this. **Note that the resultant number of samples may be higher than the set `n` due to setting of this minimum number of samples.** 
* `sampling`: the sampling strategy, e.g. 'stratified_random' where each class has a number of points proportional to its relative area, 'equal_stratified_random' where each class has the same number of points, or 'manual' which allows you to define number of samples for each class.
* `out_fname`: a filepath name for the function to export a shapefile/geojson of the sampling points into a file. You can set this to `None` if you don't need to output the file.
* `class_attr`: This is the column name of output dataframe that contains the integer class values on the classified map. 
* `drop_value`: pixel value on the classification map to be excluded from sampling.

The output of the function is a geopandas dataframe of randomly distributed points containing a column `class_attr` identifying class values. 

Here we extract around 1000 training points in total and export the points in a geojson file for use in the rest of workflow. Here we use the stratified sampling method by setting 'equal_stratified_random', but also set the minimum number of samples as 75 to avoid missing samples for some minor classes. 

As mentioned earlier we don't want the abandoned classes to be included in the samples we set drop_value as 0 before implementing the function:

In [ ]:
# Reproject to WGS84 so sample points are in lat/lon
classification_map_4326 = classification_map.rio.reproject(output_crs)
classification_map_4326.head()

In [ ]:
out_fname='training_data.geojson'
n=2100
min_sample_n=300
drop_value=0
gpd_random_samples=random_sampling(da=classification_map_4326,n=n,sampling='stratified_random',
                                   min_sample_n=min_sample_n,out_fname=None,class_attr=class_attr,drop_value=drop_value)

In [ ]:
print(f"CRS: {gpd_random_samples.crs}")
gpd_random_samples.head()

## Visualise the training data by class

In [ ]:
gpd_random_samples.explore(
    column=class_attr,
    categorical=True,
    categories=(present_classes := sorted(gpd_random_samples[class_attr].unique())),
    cmap=[colors[c] for c in present_classes],
    legend=True,
    style_kwds={"radius": 6, "fillOpacity": 0.8, "weight": 0.5}
)

## Extract Geomedian/GeoMAD values at training data point locations.

For now we run and load individual tiles and mosaic them because we have not run all tiles yet.

Once we have finalised the GeoMAD data, we will set up a GeoParquet STAC index for it. Then querying it by bbox will be simple and replace the more complicated process implemented here.

In [ ]:
# Query our GeoMAD STAC-Geoparquet by AOI bbox and datetime.
# parquet = "s3://data.ldn.auspatious.com/ausp_ls_geomad/0-0-2/ausp_ls_geomad.parquet"
stac_geoparquet = "https://s3.us-west-2.amazonaws.com/data.ldn.auspatious.com/ci_ls_geomad/0-0-2/ci_ls_geomad.parquet" # Non-Pacific
# stac_geoparquet = "https://s3.us-west-2.amazonaws.com/data.ldn.auspatious.com/ausp_ls_geomad/0-0-2/ausp_ls_geomad.parquet" # Pacific
stac_docs = search_sync(stac_geoparquet, intersects=aoi_geojson, datetime=datetime_year) # , store=S3Store(bucket="data.ldn.auspatious.com", region="us-west-2"))
print(f"Found {len(stac_docs)} STAC documents for this AOI and datetime year")
geomad_items = [Item.from_dict(feature) for feature in stac_docs]
print(f"Parsed {len(geomad_items)} STAC items")

bands = list(geomad_items[0].assets.keys())
print(f"Available bands: {bands}")

In [ ]:
# TODO: Load geomad without any CRS/res options. We want to load it as native.
geomad_ds = load(
    geomad_items,
    crs=analysis_crs,
    resolution=10, # Do 100 for faster development, but use 10m for final runs.
    groupby="solar_day",
    intersects=aoi_geojson,
    datetime=datetime_year,
    chunks={}, # Force lazy loading.
)
geomad_ds = geomad_ds.squeeze().load() # .load() to read the data into memory, .squeeze() to remove any singleton dimensions.
print(f"GeoMAD dataset shape: {geomad_ds.dims}")
geomad_ds

In [ ]:
geomad_ds.odc.explore()

In [ ]:
print(geomad_ds)

# Scale observed values

Done before calculating indices.

Apply scaling to the relevant bands -- Landsat data values are supplied as integers, but it is best practice to scale them to have values between 0 and 1 when doing calculations of indices. Landsat has a scale factor of 0.0000275 and an offset of -0.2. The cell below defines a function for this, and also sets any negative values to nan.

In [ ]:
try:
    bands.remove('count')
except ValueError:
    pass # If rerunning, count is already removed
band_names_geomad = [b for b in bands if b.endswith('mad')] # ['smad', 'bcmad', 'emad']
band_names_geomedian = [b for b in bands if b not in band_names_geomad] # ['red', 'blue', 'green', 'swir16', 'swir22', 'nir08']
print(band_names_geomad)
print(band_names_geomedian)

# Adapted from https://github.com/auspatious/cloud-native-geospatial-eo-workshop/blob/main/02_Cloud_Native_Land_Productivity_For_SDG15_LS.ipynb
def scale_offset(band):
    nodata = band == 0
    # Apply a scaling factor and offset to the band
    band = band * 0.0000275 + -0.2
    band = band.clip(0, 1)

    return band.where(~nodata, other=np.nan)

# Only scale the integer geomedian bands, not the float MAD bands
for band in band_names_geomedian:
    geomad_ds[band] = scale_offset(geomad_ds[band])

In [ ]:
# geomad_ds
print("red range: ", geomad_ds['red'].min().values, geomad_ds['red'].max().values)

# Indices

### Calculate indices like NDVI (from the Geomedian/GeoMAD bands).

In [ ]:
# Adapted from https://github.com/digitalearthpacific/dep-sdb/blob/main/src/utils.py
# Use our own function because deafrica calculate_indices is large and clunky.
def make_indices(geomad: xr.Dataset) -> xr.Dataset:
    nir = geomad.nir08
    red = geomad.red
    green = geomad.green
    blue = geomad.blue
    swir1 = geomad.swir16
    swir2 = geomad.swir22

    # Vegetation
    geomad["ndvi"] = (nir - red) / (nir + red)

    # Water
    geomad["ndwi"] = (green - nir) / (green + nir)
    geomad["mndwi"] = (green - swir1) / (green + swir1)

    # Turbidity
    geomad["ndti"] = (red - green) / (red + green)

    # Soil
    # Bare Soil Index (common form)
    geomad["bsi"] = ((swir1 + red) - (nir + blue)) / ((swir1 + red) + (nir + blue))
    # Modified Bare Soil Index
    geomad["mbi"] = ((swir1 - swir2 - nir) / (swir1 + swir2 + nir)) + 0.5

    # Built-Up
    # Built-up Area Extraction Index (BAEI)
    geomad["baei"] = (red + 0.3) / (green + swir1)
    # Built-Up Index (BUI = NDBI - NDVI)
    ndbi = (swir1 - nir) / (swir1 + nir)
    geomad["bui"] = ndbi - geomad["ndvi"]


    # Shallow water bathymetry. Need to calc it before scaling aparently
    # # Stumpf, calculate off non-scaled data to remove nan/infinities
    # geomad["stumpf"] = np.log(np.abs(geomad.green - geomad.blue)) / np.log(
    #     geomad.green + geomad.blue
    # )

    # Blue over green index
    geomad["bg"] = blue / green

    # Blue over red index
    # geomad["br"] = blue / red

    # Natural log of blue/green
    geomad["ln_bg"] = np.log(blue / green)

    # # Lyzenga... seems problematic
    # geomad["lyzenga"] = np.log(green / blue)

    return geomad

geomad_ds = make_indices(geomad_ds)
geomad_ds.head()

# Get Elevation data

In [ ]:
# Load DEM
dem_items = list(
    catalog.search(
        collections=["cop-dem-glo-30"],
        intersects=aoi_geojson,
    ).items()
)

print(f"Found {len(dem_items)} DEM items")
print(dem_items[0].to_dict() if dem_items else "NO ITEMS FOUND")

# Upscales from 30m to 10m.
dem = load(
    dem_items,
    # geobox=geomad_ds.odc.geobox, # Use the same geobox as geomad for alignment. This will reproject and resample the DEM to match geomad.
    like=geomad_ds,
    resampling="bilinear",
    patch_url=sign_url,
).squeeze().compute().rename({"data": "elevation"})

print(dem)

In [ ]:
# No need to clip or mask. It will get sampled to points anyway.
# dem = dem.rio.clip(aoi.to_crs(analysis_crs).geometry, crs=analysis_crs)
dem["elevation"].odc.explore()

# Add Geomedian, GeoMAD, and elevation data to training points.

In [ ]:
print(geomad_ds)
print(dem)

dem = dem.assign_coords(time=geomad_ds.time)
geomad_dem = xr.merge([geomad_ds, dem])
print("Red:", geomad_dem['red'].min().values, geomad_dem['red'].max().values)
print("NDVI:", geomad_dem['ndvi'].min().values, geomad_dem['ndvi'].max().values)
print("Elevation:", geomad_dem['elevation'].min().values, geomad_dem['elevation'].max().values)
print(geomad_dem)
# geomad_dem = geomad_dem.fillna(0) # Fill NaN with 0 for exploration. We will keep NaN in the actual training data.
geomad_dem.odc.explore()

In [ ]:
# Write data for model
# geomad_dem.rio.to_raster("geomad_dem.tif", dtype='uint8', compress='deflate')
geomad_dem.to_netcdf("geomad_dem.nc")

In [ ]:
band_names_with_indices = [v for v in geomad_dem.data_vars]
print(band_names_with_indices)

# Extract Geomedian and GeoMAD band values at each sample point location

# Ensure both datasets are in the same CRS (6933)
# TODO: Reproject sample points to match native CRS of geomad before sampling.
gpd_random_samples_6933 = gpd_random_samples.to_crs(analysis_crs)

# Build point-wise x/y arrays with the SAME 'points' index
points_idx = np.arange(len(gpd_random_samples_6933))
xs = xr.DataArray(gpd_random_samples_6933.geometry.x.values, dims="points", coords={"points": points_idx})
ys = xr.DataArray(gpd_random_samples_6933.geometry.y.values, dims="points", coords={"points": points_idx})
# Point-wise nearest sampling (one sampled pixel per input point)
sampled = geomad_dem[band_names_with_indices].sel(x=xs, y=ys, method="nearest")

# Convert to dataframe indexed by points; keep exactly one row per point
sampled_df = sampled.to_dataframe().reset_index()
sampled_df = sampled_df.sort_values("points").reset_index(drop=True)

# Merge back to training points (original CRS)
training_data = gpd_random_samples.reset_index(drop=True).copy()
training_data = pd.concat([training_data, sampled_df[band_names_with_indices]], axis=1)

print(f"Training data shape: {training_data.shape}")
print("Unique values per band:")
print(training_data[band_names_with_indices].nunique())
training_data.head()

## Visualise GeoMedian red values per sample point

In [ ]:
# Visualise red values using the actual data range
training_data.explore(
    column="red",
    cmap="Reds",
    k=7,
    scheme="natural_breaks",
    legend=True,
    style_kwds={"radius": 4, "fillOpacity": 0.8, "weight": 0.5},
)

## Explore elevation data on training data points

In [ ]:
training_data.explore(
    column="elevation",
    cmap="Blues",
    scheme="natural_breaks",
    k=7,
    legend=True,
    style_kwds={"radius": 4, "fillOpacity": 0.8, "weight": 0.5},
)

## Explore NDVI data on training data points

In [ ]:
training_data.explore(
    column="ndvi",
    cmap="Greens",
    scheme="quantiles",
    k=7,
    legend=True,
    style_kwds={"radius": 4, "fillOpacity": 0.8, "weight": 0.5},
)

In [ ]:
out_data = training_data.drop(columns=["time", 'spatial_ref', 'count'])

# Write the final training data
out_data.to_file(out_fname, driver="GeoJSON", index=False)
# out_data.to_file(out_fname.replace(".geojson", ".gpkg"), driver="GPKG")
# out_data.to_csv(out_fname.replace(".geojson", ".csv"), index=False)

print(f"Saved training data as geojson, CSV, and geopackage to {out_fname}")